In [1]:
from identification_suggestor import IdentificationSuggestor 
from dotenv import load_dotenv
import guidance
import os

load_dotenv()

api_key = os.getenv("api_key")
organization = os.getenv("organization")

# set the default language model used to execute guidance programs
gpt4 = guidance.llms.OpenAI(api_key=api_key, organization=organization, model="gpt-4")

c:\Users\t-rdesicilia\Documents\VSCodeProjects\Augment\askwhy_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sea_ice_variables = [
    "geopotential_heights", 
    "relative_humidity", 
    "sea_level_pressure",  
    "zonal_wind_at_10_meters", 
    "meridional_wind_at_10_meters", 
    "sensible_plus_latent_heat_flux", 
    "total_precipitation", 
    "total_cloud_cover", 
    "total_cloud_water_path",  
    "surface_net_shortwave_flux", 
    "surface_net_longwave_flux", 
    "northern_hemisphere_sea_ice_extent",
]

treatment = "surface_net_longwave_flux"
outcome = "northern_hemisphere_sea_ice_extent"

# ground truth confounders to the relationship between surface_net_longwave_flux and northern_hemisphere_sea_ice_extent
ground_truth_confounders = ["total_precipitation"]
   
ground_truth_graph = {
    ("surface_net_longwave_flux", "northern_hemisphere_sea_ice_extent"), 
    
    ("geopotential_heights", "surface_net_longwave_flux"), 
    ("geopotential_heights", "relative_humidity"), 
    ("geopotential_heights", "sea_level_pressure"), 

    ("relative_humidity", "total_cloud_cover"), 
    ("relative_humidity", "total_cloud_water_path"),
    ("relative_humidity", "total_precipitation"), 
    ("relative_humidity", "surface_net_longwave_flux"),

    ("sea_level_pressure", "relative_humidity"), 
    ("sea_level_pressure", "geopotential_heights"), 
    ("sea_level_pressure", "zonal_wind_at_10_meters"), 
    ("sea_level_pressure", "northern_hemisphere_sea_ice_extent"), 
    ("sea_level_pressure", "sensible_plus_latent_heat_flux"), 
    ("sea_level_pressure", "meridional_wind_at_10_meters"),

    ("zonal_wind_at_10_meters", "northern_hemisphere_sea_ice_extent"),
    ("zonal_wind_at_10_meters", "sensible_plus_latent_heat_flux"), 
    
    ("meridional_wind_at_10_meters", "northern_hemisphere_sea_ice_extent"),
    ("meridional_wind_at_10_meters", "sensible_plus_latent_heat_flux"), 
   
    ("sensible_plus_latent_heat_flux", "northern_hemisphere_sea_ice_extent"), 
    ("sensible_plus_latent_heat_flux", "sea_level_pressure"), 
    ("sensible_plus_latent_heat_flux", "zonal_wind_at_10_meters"), 
    ("sensible_plus_latent_heat_flux", "meridional_wind_at_10_meters"), 
    ("sensible_plus_latent_heat_flux", "total_precipitation"), 
    ("sensible_plus_latent_heat_flux", "total_cloud_cover"), 
    ("sensible_plus_latent_heat_flux", "total_cloud_water_path"), 
    
    ("total_precipitation", "northern_hemisphere_sea_ice_extent"),
    ("total_precipitation", "relative_humidity"),
    ("total_precipitation", "sensible_plus_latent_heat_flux"),
    ("total_precipitation", "surface_net_longwave_flux"),
    ("total_precipitation", "total_cloud_cover"),
    ("total_precipitation", "total_cloud_water_path"),
   
    ("total_cloud_water_path", "total_precipitation"), 
    ("total_cloud_water_path", "sensible_plus_latent_heat_flux"), 
    ("total_cloud_water_path", "relative_humidity"), 
    ("total_cloud_water_path", "surface_net_longwave_flux"), 
    ("total_cloud_water_path", "surface_net_shortwave_flux"), 
    
    ("total_cloud_cover", "total_precipitation"),
    ("total_cloud_cover", "sensible_plus_latent_heat_flux"),
    ("total_cloud_cover", "relative_humidity"),
    ("total_cloud_cover", "surface_net_longwave_flux"),
    ("total_cloud_cover", "surface_net_shortwave_flux"), 
    
    ("surface_net_shortwave_flux", "northern_hemisphere_sea_ice_extent"),
    
    ("northern_hemisphere_sea_ice_extent", "sea_level_pressure"),
    ("northern_hemisphere_sea_ice_extent", "zonal_wind_at_10_meters"),
    ("northern_hemisphere_sea_ice_extent", "meridional_wind_at_10_meters"),
    ("northern_hemisphere_sea_ice_extent", "sensible_plus_latent_heat_flux"),
    ("northern_hemisphere_sea_ice_extent", "surface_net_shortwave_flux"),
    ("northern_hemisphere_sea_ice_extent", "surface_net_longwave_flux"),
    }

In [3]:
import random 

random.seed(42)  # Set the random seed for reproducibility
subset_size = 7
selected_variables = random.sample(sea_ice_variables[:-2], subset_size)

selected_variables.append(treatment)
selected_variables.append(outcome)  


print(selected_variables)

# grab the relationships that match the subset variables
subset_graph = set()    
for a in selected_variables:
    for b in selected_variables:
        if a != b and (a, b) in ground_truth_graph:
            subset_graph.add((a, b))



['relative_humidity', 'geopotential_heights', 'meridional_wind_at_10_meters', 'surface_net_shortwave_flux', 'total_precipitation', 'sensible_plus_latent_heat_flux', 'total_cloud_water_path', 'surface_net_longwave_flux', 'northern_hemisphere_sea_ice_extent']


In [4]:
identifier = IdentificationSuggestor()

In [5]:
backdoor = identifier.suggest_backdoor(llm=gpt4, treatment=treatment, outcome=outcome, variables=selected_variables)
print("Backdoor\n", backdoor)

Stop program system You are a helpful assistant on causal reasoning and arctic sea ice and atmosphere sciences. Your goal is to answer questions factually and concisely about whether and how a causal effect can be estimated using your domain knowledge on artic sea ice and atmosphere sciences. user Is total_cloud_water_path a confounder that when conditioned on can unbias the surface_net_longwave_flux -> northern_hemisphere_sea_ice_extent causal effect estimation? 
 A. Yes
 B. No
 Let's think step-by-step to make sure that we have the right answer. Keep your argument and references to no more than one paragraph, otherwise you lose points, and wrap it within the tags, <explanation>...</explanation>. Then provide your final answer within the tags, <answer>A/B</answer>. assistant <explanation>In the context of Arctic sea ice and atmosphere sciences, total cloud water path can be a confounder because it can affect both the surface net longwave flux and the northern hemisphere sea ice extent. Clouds can modulate the surface net longwave flux by reflecting and absorbing the radiation, and the total cloud water path is a measure of the total amount of water in a column of cloud, which can change the radiation balance. On the other hand, clouds can also affect the sea ice extent by influencing the surface temperature and consequently the melting and freezing of sea ice. Therefore, conditioning on the total cloud water path can potentially unbias the causal effect estimation between surface net longwave flux and northern hemisphere sea ice extent.</explanation>

<answer>A</answer>

Backdoor
 ['relative_humidity', 'total_precipitation', 'sensible_plus_latent_heat_flux', 'total_cloud_water_path']


In [6]:
frontdoor = identifier.suggest_frontdoor(llm=gpt4, treatment=treatment, outcome=outcome, variables=selected_variables)
print("Frontdoor\n", frontdoor) 

Stop program system You are a helpful assistant on causal reasoning and arctic sea ice and atmosphere sciences. Your goal is to answer questions factually and concisely about whether and how a causal effect can be estimated using your domain knowledge on artic sea ice and atmosphere sciences. user Is total_cloud_water_path a mediator that when conditioned on can unbias the surface_net_longwave_flux -> northern_hemisphere_sea_ice_extent causal effect estimation? 
 A. Yes
 B. No
 Let's think step-by-step to make sure that we have the right answer. Keep your argument and references to no more than one paragraph, otherwise you lose points, and wrap it within the tags, <explanation>...</explanation>. Then provide your final answer within the tags, <answer>A/B</answer>. assistant <explanation>In order to determine if total_cloud_water_path can be a mediator, it needs to be influenced by the independent variable (surface_net_longwave_flux) and also have an effect on the dependent variable (northern_hemisphere_sea_ice_extent). While it is true that cloud water path can influence the amount of longwave radiation reaching the surface, thereby influencing sea ice extent, it's not directly influenced by the surface net longwave flux - rather, it's the other way around. The net longwave flux at the surface is influenced by the total cloud water path, as it impacts the amount of energy absorbed and reflected by the atmosphere. Therefore, total_cloud_water_path cannot be a mediator that when conditioned on can unbias the surface_net_longwave_flux -> northern_hemisphere_sea_ice_extent causal effect estimation. </explanation>

<answer>B</answer>

Frontdoor
 ['surface_net_shortwave_flux']


In [7]:
iv = identifier.suggest_iv(llm=gpt4, treatment=treatment, outcome=outcome, variables=selected_variables)
print("Instrumental Variable\n", iv) 

Stop program system You are a helpful assistant on causal reasoning and arctic sea ice and atmosphere sciences. Your goal is to answer questions factually and concisely about whether and how a causal effect can be estimated using your domain knowledge on artic sea ice and atmosphere sciences. user Is total_cloud_water_path an instrumental variable that when conditioned on can unbias the surface_net_longwave_flux -> northern_hemisphere_sea_ice_extent causal effect estimation? 
 A. Yes
 B. No
 Let's think step-by-step to make sure that we have the right answer. Keep your argument and references to no more than one paragraph, otherwise you lose points, and wrap it within the tags, <explanation>...</explanation>. Then provide your final answer within the tags, <answer>A/B</answer>. assistant <explanation>In order for total_cloud_water_path to be an instrumental variable that can unbiasedly estimate the causal effect of surface_net_longwave_flux on northern_hemisphere_sea_ice_extent, it must satisfy two conditions: 1) relevance - it must be correlated with surface_net_longwave_flux, and 2) exogeneity - it must not be correlated with the error term in the causal relationship between surface_net_longwave_flux and northern_hemisphere_sea_ice_extent. Total_cloud_water_path is likely correlated with surface_net_longwave_flux because clouds can influence the longwave radiation at the surface. However, it's also likely that total_cloud_water_path is influenced by other factors that can affect sea ice extent, such as temperature and wind patterns, which would violate the exogeneity condition. Therefore, it's unlikely that total_cloud_water_path can be used as an instrumental variable to unbiasedly estimate this causal effect.</explanation>

<answer>B</answer>

Instrumental Variable
 []


In [13]:
estimand = identifier.suggest_identification(llm=gpt4, variables=selected_variables, treatment=treatment, outcome=outcome, backdoor=backdoor, frontdoor=frontdoor, iv=iv)  

In [12]:
print(estimand)

Backdoor
 Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
                                     d                                        
────────────────────────────────────────────────────────────────────────────(E
d[s  u  r  f  a  c  e  _  n  e  t  _  l  o  n  g  w  a  v  e  _  f  l  u  x]  

                                                                              
[n|relative_humidity,total_precipitation,sensible_plus_latent_heat_flux,total_
                                                                              

                  
cloud_water_path])
                  
Estimand assumption 1, Unconfoundedness: If U→{s,u,r,f,a,c,e,_,n,e,t,_,l,o,n,g,w,a,v,e,_,f,l,u,x} and U→n then P(n|s,u,r,f,a,c,e,_,n,e,t,_,l,o,n,g,w,a,v,e,_,f,l,u,x,relative_humidity,total_precipitation,sensible_plus_latent_heat_flux,total_cloud_water_path,U) = P(n|s,u,r,f,a,c,e,_,n,e,t,_,l,o,n,g,w,a,v,e,_,f,l,u,x,relative_humidity,total_precipit